# 0. NSP T2A 개발환경 Set-up

In [1]:
import re
import os
import ast
import json
import time
import pandas as pd

import sqlite3
from datetime import datetime
from pprint import pprint
from glob import glob
from concurrent.futures import ThreadPoolExecutor
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# LangChain Memory: 대화의 흐름을 유지하기 위한 메모리 모듈
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory, ConversationSummaryMemory

# LangChain Chains: LLM과 연결된 체인을 생성하는 모듈
from langchain.chains import LLMChain, ConversationChain

# LangChain Core Output Parsers: LLM의 출력 값을 처리하는 파서
from langchain_core.output_parsers import StrOutputParser


##### 24-10-03 조윤재 수정 > llama3.1 사용 위해 추가 선언 필요

from openai import OpenAI
llama_api_key = 'LA-cfe00a67782746e8b1934394ee1880ca7e8f73f0adc942d984f7d421c7a5f009'

llm_llama3 = OpenAI(
api_key = llama_api_key,
base_url = "https://api.llama-api.com"
)

os.environ['OPENAI_API_KEY'] = "sk-PGVe_TzLFy4DIIbU1LxyQ-M1SjQh1lc7xRgw3KQ5RDT3BlbkFJmRvk0hkgEFM95cjiVRgFlSy4fIIj5vg4RY0fj2CS0A"
os.environ['OPENAI_API_VERSION']="2024-02-01"
os.environ['OPENAI_CHAT_DEPLOYMENT_NAME']="gpt-4o"

# 1. 사용자 프로필 기반 필터링 및 데이터 (raw data, meta data, vector DB) load

### 1-1. 사용자 프로필 기반 필터링 (TBD, Concept Only)

In [2]:
file_path = "./data/raw_data/NSP_Sample_ERP_data.csv"
df = pd.read_csv(file_path)

# 유저 메타정보 입력(혹은 불러오기)
filter_conditions = {
    'Sale_Employee': "EMP0001",        # 직원 코드
    'Sales_Organization': "Cile SVC"   # 판매 조직
}

# 필터 만들기
filter = True
for column, value in filter_conditions.items():
    filter &= (df[column] == value)

# 필터링된 데이터프레임 출력
filtered_df = df[filter]
# filtered_df

In [3]:
df

,Sales_Order_Number,Sales_Order_Date,Request_delivery_date(header-RDD),Sales_Order_Type,Sales_Order_Tax_Amount,Sales_Order_Product_Cost,Sales_Amount,Sales_Organization,Distribution_Channel,Customer,Sales_District,Payer,Sale_Employee,Sale_Employee(name),Division,Material_Group,Material_Name
0,16518916,2015-08-15,2015-08-24,Home Delivery Order,649,5076,6926,Bolivia SVC,Export2,Costco,iquique,Costco,EMP0001,Sarah Brown,Printer,Printer6,Printer62
1,10631585,2014-04-29,2014-05-03,Home Delivery Order,321,2661,3844,CH ORG,Export1,BestBuy,iquique,BestBuy,EMP0002,James Smith,AirCon,AirCon6,AirCon61
2,14946903,2023-07-06,2023-07-20,Subscription Order,713,4450,6427,Bolivia SVC,Export2,Costco,iquique,Costco,EMP0003,Heather Miller,Speaker,Speaker5,Speaker55
3,13048562,2019-05-05,2019-05-10,Subscription Order,733,4144,6309,CH ORG,Domestic2,Costco,iquique,Costco,EMP0004,James Williams,Printer,Printer9,Printer93
4,13297353,2023-07-01,2023-07-08,Normal Order,1066,6871,9913,CH ORG,Export1,Target,Metropolitan,Target,EMP0005,Michael Davis,Refg,Refg7,Refg76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,12416340,2019-02-04,2019-02-11,Normal Order,603,4089,5851,Cile SVC,Domestic1,Amazon,iquique,Amazon,EMP0049,Matthew Garcia,Monitor,Monitor5,Monitor58
29996,14583558,2018-10-23,2018-11-04,Home Delivery Order,503,4176,5858,Bolivia SVC,Export1,Amazon,iquique,Amazon,EMP0024,Jessica Rodriguez,Monitor,Monitor1,Monitor13
29997,14702992,2022-01-31,2022-02-07,Subscription Order,492,3126,4379,CH ORG,Domestic1,Amazon,Metropolitan,Amazon,EMP0097,John Rodriguez,Printer,Printer1,Printer12
29998,13678832,2023-10-10,2023-10-15,Subscription Order,731,5476,7932,Bolivia SVC,Export2,Walmart,Metropolitan,Walmart,EMP0018,Heather Smith,Speaker,Speaker1,Speaker12


In [4]:
filtered_df

,Sales_Order_Number,Sales_Order_Date,Request_delivery_date(header-RDD),Sales_Order_Type,Sales_Order_Tax_Amount,Sales_Order_Product_Cost,Sales_Amount,Sales_Organization,Distribution_Channel,Customer,Sales_District,Payer,Sale_Employee,Sale_Employee(name),Division,Material_Group,Material_Name
132,11118708,2019-04-21,2019-04-29,Subscription Order,747,5989,8986,Cile SVC,Domestic2,Costco,iquique,Costco,EMP0001,Sarah Brown,Refg,Refg2,Refg25
516,12866114,2015-07-04,2015-07-18,Home Delivery Order,656,5468,8184,Cile SVC,Domestic1,BestBuy,iquique,BestBuy,EMP0001,Sarah Brown,AirCon,AirCon7,AirCon76
711,12125378,2015-06-20,2015-06-21,Subscription Order,648,4227,5744,Cile SVC,Export1,Walmart,Metropolitan,Walmart,EMP0001,Sarah Brown,Printer,Printer8,Printer82
727,18451532,2021-03-05,2021-03-19,Home Delivery Order,159,1103,1553,Cile SVC,Domestic1,Amazon,Metropolitan,Amazon,EMP0001,Sarah Brown,Refg,Refg1,Refg17
1595,12098538,2014-02-15,2014-02-21,Subscription Order,543,3481,5273,Cile SVC,Domestic2,Costco,iquique,Costco,EMP0001,Sarah Brown,Computor,Computor8,Computor85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,18322869,2015-11-20,2015-11-28,Subscription Order,298,1662,2546,Cile SVC,Domestic1,Walmart,iquique,Walmart,EMP0001,Sarah Brown,AirCon,AirCon7,AirCon78
28979,15606055,2022-11-02,2022-11-11,Home Delivery Order,427,3339,5012,Cile SVC,Export2,Walmart,Metropolitan,Walmart,EMP0001,Sarah Brown,AirCon,AirCon6,AirCon68
29666,13390112,2016-06-05,2016-06-18,Normal Order,424,3112,4183,Cile SVC,Domestic2,Walmart,iquique,Walmart,EMP0001,Sarah Brown,Printer,Printer6,Printer64
29668,12064835,2018-07-01,2018-07-08,Normal Order,189,1522,2097,Cile SVC,Export2,BestBuy,Metropolitan,BestBuy,EMP0001,Sarah Brown,Speaker,Speaker5,Speaker52


### 고도화 필요

In [5]:
# 프로필 필터링을 통해 성능을 향상 시킬 수 있는 point 살려야 함.
# 답을 찾아갈 때 space를 좁힐 수 있는 영역이 무엇인지 확인 필요.
# 영업 사원 vs 영업 총괄
# 대화를 통해서 범위를 더 좁힐 수 있는 scenario

### 1-2. 데이터 load

In [6]:
start = time.time()
engine = sqlite3.connect(":memory:") # 가상 in-mem방식 DB 선언(SQL 확인용 engine)

##### 24-10-07 조윤재 --> 파일 교체

# CSV 파일 목록과 테이블 이름
file_paths = [
    "./data/raw_data/NSP_Sample_ERP_data.csv",
]
##### 24-10-07 조윤재 --> 테이블명 교체
table_names = ["NSP_TABLE"]

# CSV 파일을 데이터베이스에 로드
for file_path, table_name in zip(file_paths, table_names):
    df = pd.read_csv(file_path)
    df[['Sales_Order_Tax_Amount','Sales_Order_Product_Cost','Sales_Amount']] = df[['Sales_Order_Tax_Amount','Sales_Order_Product_Cost','Sales_Amount']].astype(float)
    print(df.shape)
    # SQLite 데이터베이스에 저장
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    
print("Data loading complete.")
end = time.time()

print(f"{end - start:.5f} sec")

# Dataset_path setting
meta_path = './data/meta_data/'

##### 24-10-07 조윤재 --> 파일 교체
## Table description
table_description = pd.read_csv(f'{meta_path}NSP_Table_Description_241004.csv').drop('Unnamed: 0',axis=1,errors='ignore')

## Var meta description
classified_var_meta = pd.read_csv(f'{meta_path}NSP_classified_var_meta_241007.csv').drop('Unnamed: 0',axis=1,errors='ignore')

## Value_Description
value_description = pd.read_csv(f'{meta_path}NSP_Value_Description_241007.csv').drop('Unnamed: 0',axis=1,errors='ignore')

vector_db_path = './data/vector_store/'
# openai model declaration
EMBEDDING_MODEL = "text-embedding-3-large"
# embeddings = AzureOpenAIEmbeddings(model=EMBEDDING_MODEL)  # "text-embedding-3-small" 모델 대체
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)  # "text-embedding-3-small" 모델 대체

##### 24-10-07 조윤재 --> 파일 교체
# 벡터 db 호출
user_vocab_vdb = FAISS.load_local(vector_db_path + '/nsp_vocab_241007.faiss', embeddings, allow_dangerous_deserialization=True)
user_query_vdb = FAISS.load_local(vector_db_path+'/nsp_query_241007.faiss', embeddings, allow_dangerous_deserialization=True)


(30000, 17)
Data loading complete.
0.25181 sec


# 2. 재질의 및 질문 구체화 (TBD) > LLama3.1 70b 모델 기반

### 2-1. 재질의(Re-question) Llama 모델 선언

In [7]:
llama_api_key = 'LA-cfe00a67782746e8b1934394ee1880ca7e8f73f0adc942d984f7d421c7a5f009'

llm_params = {
    "api_key": llama_api_key,
    "base_url": "https://api.llama-api.com",
    "model": "llama3.1-8b",
    'temperature': 0,
    'max_tokens' : None,  # 최대 토큰 수 제한 없음
    'timeout' : None,  # 타임아웃 설정 없음
    'max_retries' : 2,  # 최대 재시도 횟수
    }

llm_llama3 = ChatOpenAI(**llm_params)
llm_tmpt0 = llm_llama3

### 2-2. 사용자 질의 Input 및 재질의 유도

In [8]:
requestion_prompt = """
당신은 사용자의 질문에 따라 데이터를 검색하는 AI 어시스턴트입니다. 데이터를 검색하기 전에 사용자의 질문이 명확하고 구체적인지 확인해야 합니다. 다음 단계를 따라 질문을 평가하십시오:

1. **금액 또는 수량 확인:**
    - 질문에 금액 또는 수량이 명시되지 않은 경우, 기본값은 원화금액을 기준으로 조회하고 재질의하지 마시오.
    - 예시: 사용자가 "작년 실적이 어땠어?"라고 질문하면, "작년 실적을 판매원화금액 기준으로 답변드리겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.
    - 예시: 사용자가 "작년 판매량이 어땠어?"라고 질문하면, "작년 판매원화금액 전체를 답변드리겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.

2. **데이터 기간 확인:**
    - 질문에 기간이나 날짜 범위가 명시되지 않은 경우, 사용자가 이를 정의하도록 요청하십시오.
    - 예시: 사용자가 "판매 성과가 어땠나요?"라고 질문하면, "어떤 기간의 판매 성과를 분석하시려는 건가요?"라고 응답하십시오.

3.  **제품군 확인:**
    - 질문에 데이터가 제품군 관련된 데이터가 명시되지 않은 경우, 기본값은 전체 제품군으로 조회하고 재질의 하지마시오.
    - 예시: 사용자가 "작년 판매는 몇건인가요?"라고 질문하면, "작년 전체 제품군에 대한 판매건을 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.
    - 예시: 사용자가 "작년 ///제품군의 판매는 몇건인가요?"라고 질문하면, "작년 /// 제품군에 대한 판매건을 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.
    - 예시: 사용자가 "작년 ///제품군의 판매는 몇건인가요?"라고 질문하면, "작년 /// 제품군에 대한 판매건을 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.
    
4. **고객 확인:**
    - 질문에 고객이 명확히 언급되지 않은 경우, 기본값은 전체 고객 기준으로 조회하고 재질의 하지마시오.
    - 예시: 사용자가 "작년 ///제품군의 매출액은 얼마인가요?"라고 질문하면, "작년 전체 고객에게 판매된 ///제품군의 매출액에 대해 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.
    - 예시: 사용자가 "작년 판매된 금액은 얼마인가요?"라고 질문하면, "작년 고객 전체에게 판매된 총매출액에 대해 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.
    - 예시: 사용자가 "작년 판매된 수량은 얼마인가요?"라고 질문하면, "작년 고객 전체에게 판매된 수량에 대해 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.

5. **사업 부서 확인:**
    - 질문에 사업 부서가 명확히 언급되지 않은 경우, 기본값은 전체 사업 부서 기준으로 조회하고 재질의 하지마시오.
    - 예시: 사용자가 "작년 매출액은 얼마인가요?"라고 질문하면, "작년 전체 사업부 기준으로 판매된 매출액에 대해 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.
    - 예시: 사용자가 "작년 판매된 금액은 얼마인가요?"라고 질문하면, "작년 사업부서 전체 기준으로 판매된 총매출액에 대해 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.
    - 예시: 사용자가 "작년 판매된 수량은 얼마인가요?"라고 질문하면, "작년 사업부서 전체 기준으로 판매된 수량에 대해 조회하겠습니다. 잠시만 기다려 주세요."라고 응답하십시오.

6. **질문 승인:**
    - 질문에 필요한 모든 정보(금액/수량, 기간, 제품군, 고객, 사업 부서)가 포함되어 있는 경우, 추가 설명을 요청하지 않고 데이터 검색을 진행하십시오.
    - 질문 승인이 되었다면, 응답의 마지막에 질문을 이해한 내용을 정리하고, 잠시만 기다려 주세요."라는 문장으로 마무리하십시오.

### 사용 예시:
    - 사용자 질문: "작년 4분기 매출이 어떻게 되었나요?"
    - AI 응답: "작년 4분기의 매출액 기준으로 답변드리겠습니다. 잠시만 기다려 주세요."

    - 사용자 질문: "2023년 ///제품군의 전체 부서 판매량은 어땠나요?"
    - AI 응답: "2023년 전체 사업부서에서 판매된 ///제품군의 판매량을 기준으로 조회하겠습니다. 잠시만 기다려 주세요."

    - 사용자 질문: "최근 3개월 동안 특정 고객에게 판매된 제품군 중, 가장 많이 팔린 제품군의 매출은 얼마인가요?"
    - AI 응답: "최근 3개월 동안 특정 고객에게 가장 많이 팔린 제품군의 매출액에 대해 조회하겠습니다. 잠시만 기다려 주세요."

    - 사용자 질문: "지난달 칠레에서 어떤 제품군들이 판매되었나요?"
    - AI 응답: "지난달 칠레에서 판매된 제품군 전체 리스트를 알려드리겠습니다. 잠시만 기다려 주세요."

    - 사용자 질문: "매출액 알려줘."
    - AI 응답: "어떤 기간의 매출액을 조회하시려는 건가요?"

    - 사용자 질문: "제품군 매출이 어땠어?"
    - AI 응답: "어떤 기간동안 어떤 제품군의 매출을 조회하시려는 건가요?"

    - 사용자 질문: "고객 매출이 어땠어?"
    - AI 응답: "어떤 기간동안 어떤 고객의 매출을 조회하시려는 건가요?"

    - 사용자 질문: "사업부 매출이 어땠어?"
    - AI 응답: "어떤 기간동안 어떤 사업부서의 매출을 조회하시려는 건가요?"

### 구현:
AI 어시스턴트는 위의 로직을 사용하여 모든 사용자 질문이 충분히 구체적이고 상세한지를 확인한 후에만 데이터 검색을 진행합니다. 중요한 세부 사항(금액/수량, 기간, 제품군, 고객, 사업 부서)이 누락된 경우, AI는 추가 설명을 요청한 후에 검색을 진행합니다.

### 추가 정보:
- 금년도, 올해, 이번년도, 오늘은 2024년입니다. 작년은 2023년, 내년은 2025년입니다. 최종답변에는 날짜를 숫자로 응답하십시오.
- 당신은 삼성전자 회사의 ERP 데이터를 검색하는 AI 어시스턴트입니다.(단 사용자가 삼성전자를 언급할때만 언급하시오.)  

### 주의 사항:
- 질문 승인이 되었다면, 응답의 마지막에 질문을 이해한 내용을 정리하고, 잠시만 기다려 주세요."라는 문장으로 마무리하십시오.
"""

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(requestion_prompt),
        HumanMessagePromptTemplate.from_template("{history}"),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

memory = ConversationSummaryMemory(llm=llm_tmpt0, max_token_limit=1000) # memory 선언

chain = ConversationChain(
    llm = llm_tmpt0,
    prompt = chat_prompt,
    input_key="user_input",
    memory = memory,
    output_parser = StrOutputParser(),
) # LLM 체인 선언


In [9]:
question_lst = [
    "최근 10년간 연도별 총 판매금 알려줘"
    "2020 ~ 2023년동안 각 제품 그룹별로 총금액 알려줘"
    "사업부서별로 판매하는 제품군 리스트 알려줘"
    "모니터 제품군에 있는 제품리스트 알려줘"
    "금액기준으로 2023년 가장 많이 팔린 제품 알려줘"
    "최근 10년간 고객별로 연도별 총 판매금 알려줘"
    "2023년 각 고객별로 내수와 수출 판매의 비중 알려줘"
    "영업조직별로 판매금액과 판매건수 알려줘"
    "각 제품그룹별로 마진률의 평균을 알려줘"
    "최근 1년 주문고객과 물건받는곳이 달랐던 건수와 금액 알려줘"
    "전체 기간동안 가장 많은 금액을 판매한 고객은 어떤 제품을 제일 많이 샀니?"
]

In [10]:

question = "각 제품그룹별로 마진률의 평균을 알려줘"

answer = chain.invoke({"user_input": question}) # 실행
pprint(answer["response"]) # 출력


('각 제품그룹별로 마진률의 평균을 알려드리겠습니다. 잠시만 기다려 주세요.\n'
 '\n'
 '먼저, 제품그룹이 어떤 것인지 확인해 드리겠습니다. 삼성전자 회사의 제품그룹은 다음과 같습니다.\n'
 '\n'
 '- 디스플레이\n'
 '- 메모리\n'
 '- 시스템 LSI\n'
 '- 세미컨덕터\n'
 '- 모바일 디바이스\n'
 '- 디스플레이 솔루션\n'
 '- 모바일 커넥티드\n'
 '-')


### 2-3. feedback 기반 재질의 항목 입력

In [11]:
feedback = "각 제품그룹별로 마진률의 평균을 알려줘"

new_answer = chain.invoke({"user_input": feedback}) # 실행 (재질의)

# new_answer의 response를 확인하여 조건에 맞는지 검사
if new_answer["response"].strip().endswith("잠시만 기다려 주세요."):
    final_answer = new_answer["response"]  # 질문을 final_answer 변수에 반영
else:
    final_answer = None  # 조건이 맞지 않으면 final_answer를 None으로 설정하거나 다른 처리

pprint(final_answer)

None


### 2-4. Schema Link 용 정제 질문 (Refined Question) 생성

In [12]:
refine_prompt = """
역할 설명:
당신은 사용자가 제공한 정보를 바탕으로, 데이터 조회에 필요한 질문을 명확하고 간결한 문장으로 재구성하는 AI 어시스턴트입니다. 이 과정에서 필요한 모든 정보를 유지하고, 불필요한 세부 사항은 제거하여 최종적으로 효율적이고 직관적인 질문을 만들어냅니다.
c
작업 지침:
핵심 정보 유지: 질문에서 필수적인 요소(예: 금액/수량, 기간, 제품군, 고객, 사업 부서)는 반드시 포함하십시오. 이는 데이터 조회에 있어 핵심적인 부분입니다.
불필요한 정보 제거: 설명이나 중복된 정보를 제거하고, 질문의 목적에 직접적으로 관련된 정보만 남기십시오.
간결성과 명확성: 질문을 가능한 한 간결하고 명확하게 표현하십시오. 사용자가 원하는 데이터를 신속하게 조회할 수 있도록 돕는 것이 목표입니다.
사용자의 의도 반영: 최종 질문은 사용자가 알고자 하는 내용을 정확하게 반영해야 합니다.

예시:
Input: '감사합니다. 이제 질문을 명확히 이해했습니다.'
'1. 판매 금액:'
' - 2024년 1월부터 6월까지 삼성전자의 판매 금액을 알고 싶으신 거죠?'
'2. 설비 증설:'
' - 2024년 1월부터 6월까지 삼성전자의 설비 증설에 대한 전반적인 정보를 원하시는 거죠?'
'이제 데이터를 검색하여 해당 정보를 제공하겠습니다.'

Output: 2024년 1월부터 6월까지 삼성전자의 판매 금액과 설비 증설 정보를 제공해줘.

Input: '질문에 필요한 모든 정보가 포함되어 있으므로, 데이터 검색을 진행하겠습니다.'
'지난 10년간 각 연도별 판매 금액과 거래업체 수를 그래프로 그리기 위해 필요한 데이터를 수집하고 분석하겠습니다.'

Output: 지난 10년간 각 연도별 판매 금액과 거래업체 수 데이터를 그래프로 제공해줘.

Input: '감사합니다. 이제 질문이 명확해졌습니다. 다음과 같이 요약할 수 있습니다:'
'- 금액/수량: 원화 금액'
'- 기간: 2018년'
'- 항목: 판매 금액 상위 10개 Material Group'
'- 신규 거래업체: 2024년에 처음 거래를 시작한 업체'
'이제 요청하신 데이터를 검색하겠습니다.'

Output: 2018년 동안 모든 부서에서 판매 금액이 상위 10개인 Material Group과 2024년에 처음 거래를 시작한 업체 정보를 제공해줘.

Input: ('감사합니다. 이제 질문이 명확해졌습니다. 요청하신 내용을 정리하면 다음과 같습니다:'
'- 신규거래업체: 2024년에 처음 거래를 시작한 업체'
'- 판매 금액: 전체 조직의 판매 금액'
'- 기간: 2019년부터 2023년까지'
'이제 2024년에 처음 거래를 시작한 업체 중 거래금액 상위 5개 업체의 2019년부터 2023년까지의 판매 금액을 그래프로 그리겠습니다. 데이터'
'를 검색하고 분석하는 데 시간이 조금 걸릴 수 있습니다.')

Output: 2019년부터 2023년까지의 판매 금액과 2024년에 처음 거래를 시작한 상위 5개 업체의 정보를 제공해줘.

Input: ('알겠습니다. 요청하신 내용을 다시 정리하면 다음과 같습니다:'
'- 금액 또는 수량 확인: 판매요청한 것 중에서 판매 금액과 차이가 큰 상위 10개의 판매요청서를 원하십니다.'
'- 데이터 기간 확인: 2023년 1월 1일부터 2023년 12월 31일까지의 데이터를 월별로 원하십니다.'
'- 조직 명확성 확인: 전체 조직의 데이터를 원하십니다.'
'이제 요청하신 데이터를 검색하여 표로 작성하겠습니다.')

Output: 2023년 동안의 판매요청 금액과 실제 판매 금액의 차이가 큰 상위 10개의 판매요청서를 월별로 표로 제공해줘.

Input: '알겠습니다. 요약하자면, 2024년 동안의 입고 건 중에서 발주단가와 입고단가가 상이한 건수를 입고담당자별로 알고 싶으신 거군요. 이제 데이터를 검색하여 요청하신 정보를 제공하겠습니다.'

Output: 2024년 동안 발주단가와 입고단가가 다른 건수를 입고담당자별로 제공해줘.

"""


# 프롬프트를 생성할 때 메모리 입력값과 일치하는 변수를 사용
refine_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(refine_prompt),
        HumanMessagePromptTemplate.from_template("{final_answer_input}"),  # 사용자 입력
    ]
)

refine_chain = refine_prompt | llm_tmpt0 | StrOutputParser() # final asnwer generator chain 선언

refined_question = refine_chain.invoke({"final_answer_input": final_answer}) # 실행

refined_question # 출력

'질문이 없습니다. 질문을 입력해 주시면, 데이터 조회에 필요한 질문을 명확하고 간결한 문장으로 재구성해 드리겠습니다.'

# 3. Schema Link

### 3-1. Schema Link 용 Llama3.1 모델 선언

In [13]:
llama_api_key = 'LA-cfe00a67782746e8b1934394ee1880ca7e8f73f0adc942d984f7d421c7a5f009'

llm_llama3 = OpenAI(
api_key = llama_api_key,
base_url = "https://api.llama-api.com"
)

In [14]:
# 내부 환경에서는 호출 횟수에도 제한이 있다.
# 당신은 이렇게 질문했는데, 우리는 이렇게 했다. 3개월으로 했는데 기간은 어떻게 변경하시겠냐

### 3-2. Vector DB 내 유사 few-shot 탐색 및 유사도 판별

In [15]:
"""
1 : few-shot DB 내 완전 일치 case 존재 > LLM 기반 schema link 단계 skip
0 : few-shot DB 내 유사 case 존재 > 검색 결과 + LLM 기반 추가 schema link
-1 : 유사 case 없음 > LLM 기반 schema link
"""

'\n1 : few-shot DB 내 완전 일치 case 존재 > LLM 기반 schema link 단계 skip\n0 : few-shot DB 내 유사 case 존재 > 검색 결과 + LLM 기반 추가 schema link\n-1 : 유사 case 없음 > LLM 기반 schema link\n'

In [16]:

"""
내부 포팅 작업
1. python interperter 변경 (자체 모델 3.10 기반, 삼성 개발 환경 3.12 기반) : 개발 환경 재구축 (interpereter <-> 패키지 호환성 검증 및 수정)
2. 구축 개발 환경 내 자체 모델 코드 이관 및 trouble shooting : interpreter, 패키지 version 변경 시 발생하는 error & in/output 수정 필요
3. 기준 LLM 변경 : llama3.1 70b (api) --> llama3.1 405b (삼성 on-pre), 코드 및 프롬프트 수정
4. 기준 embedding 모델 변경 : 자체 모델 embedding model (openai api) --> 삼성 환경 embedding model 변경
5. vector DB 변경 : langchain FAISS (api) --> 삼성 환경 내 적용 가능한 vector DB 변경 및 적용
6. 검증 및 안정화 기간 :
"""


'\n내부 포팅 작업\n1. python interperter 변경 (자체 모델 3.10 기반, 삼성 개발 환경 3.12 기반) : 개발 환경 재구축 (interpereter <-> 패키지 호환성 검증 및 수정)\n2. 구축 개발 환경 내 자체 모델 코드 이관 및 trouble shooting : interpreter, 패키지 version 변경 시 발생하는 error & in/output 수정 필요\n3. 기준 LLM 변경 : llama3.1 70b (api) --> llama3.1 405b (삼성 on-pre), 코드 및 프롬프트 수정\n4. 기준 embedding 모델 변경 : 자체 모델 embedding model (openai api) --> 삼성 환경 embedding model 변경\n5. vector DB 변경 : langchain FAISS (api) --> 삼성 환경 내 적용 가능한 vector DB 변경 및 적용\n6. 검증 및 안정화 기간 :\n'

In [ ]:
print("-----------------------------------------------------------------------------------------------")
print(refined_question)

'''
과거 자료를 바탕으로 Field 찾도록
- User_query history기준으로 Field 찾기

'''
user_query_rag_rslt = user_query_vdb.similarity_search(refined_question, k=3)

user_query_rag_rslt_string = '\n'.join([f"""
                                        ### 추천 예시{idx+1}
                                        ## 사용자 질의 : {doc.page_content}
                                        ### 사용한 FIELDS: {doc.metadata['FIELDS']}
                                        """
                                        for idx, doc in enumerate(user_query_rag_rslt)])


##### 24-10-03 조윤재 수정 > 변수명 변경해줬음 (template --> template_00), f string으로 변경해줬음
template_00 = f"""
        # 역할:
        당신은 데이터 분석 및 SQL 전문가로서, 사용자 질의에 대한 데이터 추출을 위해 필요한 FIELD를 도출하는 임무를 수행합니다.
        
        # 목적:
        [사용자 질의]에 적합한 FIELD를 식별하고, [사용자 추천]에서 유사한 질의에 사용된 필드들을 기반으로 최종적으로 필요한 필드를 도출합니다.

        # 절차:
        ##1. 유사성 평가: [사용자 질의]와 [사용자 추천]의 세 가지 질의 간의 유사성을 평가합니다.
        - "동일": [사용자 질의]와 [사용자 추천]의 질의가 완전히 일치할 경우.
        - "보통": [사용자 질의]와 [사용자 추천]의 질의가 일부 유사할 경우.
        - "다름": [사용자 질의]와 [사용자 추천]의 질의가 일치하지 않을 경우.
        
        ##2.필드 선택: 유사성 평가 결과에 따라 필드를 선택합니다.
        - "동일" (분류결과: 1): [사용자 추천]에서 사용된 모든 FIELD를 포함하여 출력합니다.
        - "보통" (분류결과: 0): 유사한 단어 혹은 문장에 해당하는 FIELD만 선택적으로 포함하여 출력합니다.
        - "다름" (분류결과: -1): FIELD를 포함하지 않고, 분류 결과만 출력합니다.
        
        ##3. 주의사항: 특정 조건에 따라 필수적으로 포함해야 할 필드가 있습니다.
        - 모든 경우:
            필수 필드: NSP_TABLE.Sales_Order_Date
        - 금액 관련 단어가 포함된 경우:
            필수 필드: {classified_var_meta[classified_var_meta['LV1'] == '금액 (Amount)'].FIELD.to_list()}
        - 제품 관련 단어가 포함된 경우:
            필수 필드: NSP_TABLE.Material_Description
        - 고객 관련 단어가 포함된 경우:
            필수 필드: NSP_TABLE.Customer_Name
        - 영업 조직 관련 단어가 포함된 경우:
            필수 필드 : NSP_TABLE.Sales_Organization
        - 사업 부서 관련 단어가 포함된 경우:
            필수 필드 : NSP_TABLE.Division_Description
        
        # 주어진 정보:
        ** [사용자 질의] : {refined_question}
        ** [사용자 추천] : 사용자(질의자)가 과거에 사용했던 사용자 질의 및 사용된 FIELD들의 매칭
            {user_query_rag_rslt_string}
        
        ** output format : Check the output format below and output it in Python dictionary format.
        ** caution : Please do not include commentary or additional explanations in your answer.
        ** output format example
            ```python
            {{  
                    "분류결과" : 0,
                    "FIELDS" : "NSP_TABLE.Sales_Order_date, NSP_TABLE.Sales_Amount, NSP_TABLE.Material_Description, NSP_TABLE.Distribution_Channel_Description"
            }}
            ```
        """

##### 24-10-03 조윤재 수정 > llama3로 변경
result = llm_llama3.chat.completions.create(
model="llama3.1-70b",
messages=[
    {"role": "system", "content": "당신은 데이터 분석 및 SQL 전문가로서, 사용자 질의에 대한 데이터 추출을 위해 필요한 FIELD를 도출하는 임무를 수행합니다."},
    {"role": "user", "content": template_00}
],

)

step00_result = result.choices[0].message.content
step00_result = step00_result.split('python')[-1]
step00_result_dict = ast.literal_eval(step00_result.replace('```','').replace('\n',''))


classification_criteria = step00_result_dict['분류결과']
field_set = set()

# 과거 History 기반으로 질의내역이 존재하는 항목에 대해서는 VectorDB에서 검색 후 criteria 0 반환, 조건 미달 시 하단의 로직 추가 실행
if (type(classification_criteria) != int) or (classification_criteria > 1) or (classification_criteria < -1):
    classification_criteria = -1

In [ ]:
classification_criteria

In [16]:
classification_criteria = 0 # 유사 case 존재 가정

### 3-3. criteria 결과 기반 > LLM 기반 schema link (Entity Recognition 및 link)

In [22]:
llm_llama3_01 = OpenAI(
api_key = llama_api_key,
base_url = "https://api.llama-api.com"
)

llm_llama3_02 = OpenAI(
api_key = llama_api_key,
base_url = "https://api.llama-api.com"
)


In [ ]:
############################################################################################################################################################################
'''
사용자 질의가 기존질문과 동일한 경우 Vector DB에서 Field를 가져와 Schema linking 결과로 출력함
즉 GPT 추론과정을 진행하지 않음
'''
if int(classification_criteria) ==  1:
    
    print('SEARCHING BY HISTORY')
    [field_set.add(x) for x in step00_result_dict['FIELDS'].split(',')]
    print(f"유사 few-shot 검색을 통해 확보한 FIELD 리스트: {field_set}")
    
else:
    ############################################################################################################################################################################
    if int(classification_criteria) == 0:
        print('SEARCHING BY HYBRID')
        # [table_set.add(x) for x in step00_result_dict['TABLE']] # (김광률 수정 240826)
        [field_set.add(x) for x in step00_result_dict['FIELDS'].split(',')]
        print(f"유사 few-shot 검색을 통해 확보한 FIELD 리스트: {field_set}")
        
    else :
        print('SEARCHING BY LLama3.1 70b')
        
        
    ############################################################################################################################################################################
    step01_entity_extract = f"""
    # 역할: 
    당신은 데이터 분석 및 SQL 작성 분야의 전문가입니다.

    # 목적:
    주어진 문장에서 SQL 쿼리를 작성하기 위해 필요한 형태소(Entity)를 추출하고, 이를 바탕으로 후보 FIELD를 식별합니다.

    # 절차:
    1. 문장 분석: 문장을 형태소 단위로 분해하고 각 형태소의 역할을 파악합니다.
    2. 핵심 정보 추출: SQL 쿼리 작성에 필요한 핵심 정보를 추출합니다. 주로 명사, 날짜, 고유명사 등이 포함됩니다.
    3. SQL 쿼리 구조 파악: 추출된 정보를 바탕으로 SQL 쿼리의 구조를 설계합니다.
    
    # 주어진 문장:
    {question}

    # 주의사항:
    - 추출된 형태소 및 단어 중 SAP-ERP 테이블의 컬럼 이름(필드)이 될 수 있는 단어/형태소들만 출력하십시오.
    - 모든 약어 및 축약어는 반드시 풀어쓰십시오.
    - 한국어와 영어 모두 주어진 예시와 같은 형태로 출력하십시오.
    
    # 출력 예시:
    "한국어(영어), 한국어(영어), 한국어(영어), ..."

    # 출력 지침:
    최종 결과만 출력하십시오.
    """

    ##### 24-10-03 조윤재 수정 > llama3로 변경

    result_01 = llm_llama3.chat.completions.create(
    model="llama3.1-70b",
    messages=[
        {"role": "system", "content": "당신은 데이터 분석 및 SQL 작성 분야의 전문가입니다. 주어진 문장에서 SQL 쿼리를 작성하기 위해 필요한 형태소(Entity)를 추출하고, 이를 바탕으로 후보 FIELD를 식별합니다."},
        {"role": "user", "content": step01_entity_extract}
    ],)

    result_01 = result_01.choices[0].message.content
    splited_result_01 = [x.strip() for x in result_01.split(',')]

    ###################################개체별 속성 추론#####################################################################################################
    ##### 24-10-07 조윤재 --> 테이블명 교체
    lv0_classification_criteria = f"""
    1. **Sales ans Distribution History**
        - {table_description[table_description['Table'] == 'NSP_TABLE']['Table Description'].values[0]}
    """


    lv1_classification_criteria = f""" 
    1. **날짜 (Date)**
        - 날짜와 관련된 필드들
        - 예: {classified_var_meta[['TABLE','FIELD','DESCRIPTION','LV0','LV1']][classified_var_meta['LV1'] == '날짜 (Date)'].FIELD.to_list()}

    2. **텍스트 (Text)**
        - 설명이나 텍스트 필드들
        - 예: {classified_var_meta[['TABLE','FIELD','DESCRIPTION','LV0','LV1']][classified_var_meta['LV1'] == '텍스트 (Text)'].FIELD.to_list()}

    3. **객체 (Object)**
        - 특정 객체나 구조와 관련된 필드들
        - 예: {classified_var_meta[['TABLE','FIELD','DESCRIPTION','LV0','LV1']][classified_var_meta['LV1'].isin(['조회대상 (Reference)','객체 (Object)'])].head(3).FIELD.to_list()}

    4. **금액 (Amount)**
        - 금액과 관련된 필드들
        - 예: {classified_var_meta[['TABLE','FIELD','DESCRIPTION','LV0','LV1']][classified_var_meta['LV1'] == '금액 (Amount)'].FIELD.to_list()}

    5. **사람 (Person)**
        - 사람과 관련된 필드들
        - 예: {classified_var_meta[['TABLE','FIELD','DESCRIPTION','LV0','LV1']][classified_var_meta['LV1'] == '사람 (Person)'].head(3).FIELD.to_list()}
    """

    step02_lv_classification = f"""
    # 목적:
    주어진 [lv0 분류기준]을 사용하여 [사용자 질의]를 분류하고, [lv1 분류기준]을 활용해 [추출된 엔티티 리스트]의 각 요소를 분류합니다.

    # 절차:
    1. [사용자 질의]를 분석하고 이해합니다.
    2. 이해한 내용을 바탕으로 [lv0 분류기준]에 따라 [사용자 질의]를 LV0 수준에서 분류합니다.
    3. [추출된 엔티티 리스트]의 각 요소를 [lv1 분류기준]에 따라 LV1 수준에서 분류합니다.
    
    # 주어진 정보:
    - 사용자 질의: {refined_question}
    - 추출된 엔티티 리스트: {result_01}
    - lv0 분류기준: {lv0_classification_criteria}
    - lv1 분류기준: {lv1_classification_criteria}
    
    # caution 1 : Please do not include commentary or additional explanations in your answer.
    # caution 2 : Do not omit punctuation marks in output format.
    # caution 3 : Do not stop in the middle of your answer and print it out to the end according to the format.

    # output format : Check the output format below and output it in Python dictionary format.
    # output format example

    {{
    "사용자 질의(LV0)": "Sales and Distribution History",
    "엔티티(LV1)": {{
        "구매 요청자": "사람 (Person)",
        "2024년": "날짜 (Date)",
        ...
        "구매 요청": "객체 (Object)",
        }}
    }}
    """

    
    ##### 24-10-03 조윤재 수정 > llama3로 변경, step02_lv_classification 프롬프트 부분에서 출력 방법 설명 부분 일부 갱신
    ##### 출력 방법:
    ##### 부연 설명을 제외한 최종 결과만 아래의 출력 예시를 참고하여 Python 딕셔너리 형태로 출력하십시오.

    result_02 = llm_llama3_01.chat.completions.create(
    model="llama3.1-70b",
    messages=[
        {"role": "system", "content": "당신은 데이터 분석 및 SQL 작성 분야의 전문가입니다. 주어진 [lv0 분류기준]을 사용하여 [사용자 질의]를 분류하고, [lv1 분류기준]을 활용해 [추출된 엔티티 리스트]의 각 요소를 분류합니다."},
        {"role": "user", "content": step02_lv_classification}
    ],)
    result_02 = result_02.choices[0].message.content
    if result_02.strip()[-1] != '}':
        result_02 = result_02 + '}}'
    step02_result_dict = ast.literal_eval(result_02.replace('python','').replace('```',''))
    ###################################개체명 & 속성 추론 결과에 대한 Table & Field 맵핑#####################################################################################################

    # 사용자 질의(LV0)와 엔티티(LV1)를 설정
    lv0 = step02_result_dict["사용자 질의(LV0)"]
    lv0 = [lv0] if type(lv0) != list else lv0
    entitys_dict = step02_result_dict["엔티티(LV1)"]

    # Entity 목록을 미리 준비
    entity_list = list(entitys_dict.keys())

    # FAISS를 활용한 배치 검색 - 엔티티마다 검색 결과를 저장
    user_vocab_rag_rslt_dict = {}
    for entity in entity_list:
        user_vocab_rag_rslt = user_vocab_vdb.similarity_search(entity)
        user_vocab_rag_rslt_dict[entity] = user_vocab_rag_rslt

    # Table과 Field를 위한 집합 초기화
    field_set = set()

    # 스레드를 사용한 병렬 처리
    def process_entity(entity):
        # 미리 가져온 검색 결과 사용
        user_vocab_rag_rslt = user_vocab_rag_rslt_dict[entity]
        user_vocab_rag_rslt_string = '\n'.join([f"Entity: {doc.page_content} - FIELD: {doc.metadata['FIELD']}" for doc in user_vocab_rag_rslt])
        
        # lv1을 리스트로 변환 (단일 항목을 리스트로 처리)
        lv1 = entitys_dict[entity]
        lv1 = [lv1] if type(lv1) != list else lv1
        
        # Pandas 필터링 작업
        sliced_var_meta = classified_var_meta[(classified_var_meta['LV0'].isin(lv0)) & (classified_var_meta['LV1'].isin(lv1))]
        sliced_var_meta_input = sliced_var_meta[['TABLE', 'FIELD', 'DESCRIPTION', 'DATA_TYPE']]
        
        table_list = table_description['Table'].to_list()
        
        # LLM 프롬프트 설정 및 실행
        step03_field_searching = f"""
        # 역할:
        당신은 데이터 분석 및 SQL 전문가입니다.

        # 목적:
        요청받은 사용자 질의를 처리하기 위해 질의 내 [엔티티]를 [후보 Field] 간 매칭하려고 합니다.

        # 절차:
        1. 문맥 이해: [사용자 질의]의 맥락을 파악하고, [엔티티]의 의미를 확인합니다.
        2. 필드 매칭: [후보 Field]에 포함된 Field를 확인하여, [엔티티]와 매칭될 수 있는 필드를 가능한 많이 선택합니다 (다중 선택 가능).
        3. 사용자 추천 참고: [사용자 추천] 정보는 사용자가 과거에 사용했던 엔티티와 필드들의 매칭 정보를 포함하고 있으므로, 이를 확인하여 최종 선택에 참고합니다.
        4. 테이블 선택: 앞서 선택된 필드와 관련된 {table_list} 내의 테이블을 선택합니다.
        5. 결과 출력: 아래 출력 방법에 따라 결과를 출력합니다.
        
        # 주어진 정보:
        - 후보 Field: 엔티티와 매칭될 수 있는 필드 후보들의 설명 (마크다운 형식): {sliced_var_meta_input}
        - 사용자 질의: {refined_question}
        엔티티: {entity}
        사용자 추천: 사용자가 과거에 사용했던 엔티티와 필드들의 매칭 {user_vocab_rag_rslt_string}

        # caution 1 : Be sure to match accurately with a FIELD within the candidate field range.
        # caution 2 : Please do not include commentary or additional explanations in your answer.
        # caution 3 : Do not omit punctuation marks in output format.
        # caution 4 : Do not stop in the middle of your answer and print it out to the end according to the format.

        # output format : Check the output format below and output it in Python dictionary format.
        # output format example

            ```python
            {{  
                    "FIELDS" : "NSP_TABLE.Sales_Order_date, NSP_TABLE.Sales_Amount, NSP_TABLE.Material_Description, NSP_TABLE.Distribution_Channel_Description"
            }}
            ```     
        """
        ##### 24-10-03 조윤재 수정 > llama3로 변경, step03_field_searching 프롬프트 부분에서 출력 방법 설명 부분 일부 갱신
        ##### 출력 방법:
        ##### 부연 설명을 제외한 최종 결과만 아래의 출력 예시를 참고하여 Python 딕셔너리 형태로 출력하십시오.
        
        result_03 = llm_llama3_02.chat.completions.create(
        model="llama3.1-70b",
        messages=[
            {"role": "system", "content": "당신은 데이터 분석 및 SQL 전문가입니다. 요청받은 사용자 질의를 처리하기 위해 질의 내 [엔티티]를 [후보 Field] 간 매칭하려고 합니다."},
            {"role": "user", "content": step03_field_searching}
        ],)
        result_03 = result_03.choices[0].message.content
        chk_03 = result_03    
        match = re.search(r'\{\s*"FIELDS"[^}]*\}', result_03)
        json_str = match.group()
        cleaned_str = json_str.replace('python', '').replace('```', '')

        
        # 결과를 파싱하여 Python dictionary로 변환
        step03_result_dict = ast.literal_eval(cleaned_str)
        
        # step03_result_dict = ast.literal_eval(result_03.replace('python', '').replace('```', ''))
        return step03_result_dict

    # ThreadPoolExecutor를 사용한 병렬 처리
    if __name__ == "__main__":
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(process_entity, entity_list))

        # 각 스레드의 결과를 집합에 추가
        for step03_result_dict in results:            
            if isinstance(step03_result_dict['FIELDS'], list):
                field_set.update(step03_result_dict['FIELDS'])
            else:
                field_set.add(step03_result_dict['FIELDS'])
           

# 최종 결과로 사용될 집합들을 리스트로 변환
field_list_result = list(field_set)
print(f"LLM을 통해 NER, 비즈니스 용어 사전 탐색을 통해 생성한 후보 FIELD 추가: {step03_result_dict}")

# 최종 결과 출력
print(f"FIELD 리스트: {field_list_result}")

#################################################과거 질의응답 내역 기반 맵핑 결과 추가 보강###################################################################
user_query_rag_rslt = user_query_vdb.similarity_search(refined_question, k=3)
user_query_rag_rslt_string = '\n'.join([f"##{idx+1} 사용자 질의 : {doc.page_content} \n사용한 FIELDS: {doc.metadata['FIELDS']}"
                                         for idx,doc in enumerate(user_query_rag_rslt)])


given_info_4 = f"""
** 주어진 정보 **
1) [사용자 질의] : {refined_question}
2) [사용자 추천] : 사용자(질의자)가 과거에 사용했던 사용자 질의 및 사용된 Field들의 매칭
    {user_query_rag_rslt_string}
"""

step04_fnl_linking = f"""
        당신은 데이터 분석 및 SQL 작성 분야의 전문가입니다. 사용자의 요청에 따라 데이터를 추출하기 위해 필요한 FIELD를 식별합니다.
        
        ** 목적 : 당신의 지식을 활용하여 [사용자 질의]에 대답하기 위해 필요한 Field들을 [사용자 추천]내 Field들에서 찾아주세요.
        ** 절차 : 
            1) [사용자 질의]와 주어진 정보 중 [사용자 추천]의 사용자 질의가 유사한지 확인해보세요.
            2) #1의 결과로 [사용자 추천]의 사용자 질의와 [사용자 질의]가 같거나 충분히 유사한 경우에는 모든 Field를 포함합니다.
            3) 이외 추가할 수 있는 Field들을 [사용자 추천]에서 찾아서 최종출력하세요.
        
        ** 주의사항
            1) 모든 [사용자 질의]는 아래 Field들이 필수로 포함되어야함.
            - NSP_TABLE.Sales_Order_Date
            2) [사용자 질의]에 금액 관련 단어가 포함되면 아래 Field들은 필수로 포함되어야함.
            - {classified_var_meta[classified_var_meta['LV1'] == '금액 (Amount)'].FIELD.to_list()}
            3) [사용자 질의]에 제품 관련 단어가 포함되면 아래 Field들은 필수로 포함되어야함.
            - NSP_TABLE.Material_Description
            4) [사용자 질의]에 고객 관련 단어가 포함되면 아래 Field들은 필수로 포함되어야함.
            - NSP_TABLE.Customer_Name
            5) [사용자 질의]에 영업 조직 관련 단어가 포함되면 아래 Field들은 필수로 포함되어야함.
            - NSP_TABLE.Sales_Organization
            6) [사용자 질의]에 사업 부서 관련 단어가 포함되면 아래 Field들은 필수로 포함되어야함.
            - NSP_TABLE.Division_Description

        ** 주어진 정보
            1) [사용자 질의] : {refined_question}
            2) [사용자 추천] : 사용자(질의자)가 과거에 사용했던 사용자 질의 및 사용된 Field들의 매칭
                {user_query_rag_rslt_string}

        ** 최종 결과만 출력하시오
        ** 출력방법 : 최종결과만 python dictionary형태로 출력
            출력예시)
            ```python
            {{  
                    "FIELDS" : "NSP_TABLE.Sales_Order_date, NSP_TABLE.Sales_Amount, NSP_TABLE.Material_Description, NSP_TABLE.Distribution_Channel_Description"
            }}
            ```              
            
        """

##### 24-10-03 조윤재 수정 > llama3로 변경
result_04 = llm_llama3.chat.completions.create(
model="llama3.1-70b",
messages=[
    {"role": "system", "content": "당신은 데이터 분석 및 SQL 작성 분야의 전문가입니다. 사용자의 요청에 따라 데이터를 추출하기 위해 필요한 FIELD를 식별합니다."},
    {"role": "user", "content": step04_fnl_linking}
],)
result_04 = result_04.choices[0].message.content    

step04_result_dict = ast.literal_eval(result_04.replace('python','').replace('```',''))


### final result
# [field_set.add(x) for x in step04_result_dict['FIELD']]
[field_set.add(x) for x in step04_result_dict['FIELDS'].split(',')]

field_list = list(field_set)

field_string = ', '.join(field_list)

print(f"질의 : {refined_question}")
print(field_string)
print("-----------------------------------------------------------------------------------------------")
print("\n")

### 3-4. Hallucination filltering (FIELD 메타 기반)

In [ ]:
"""
DB 내 존재하지 않은 TABLE & FIELD 명은 제외함
"""
table_final = pd.DataFrame()
schema_final = []
for table_field in set([x.strip() for x in field_string.split(',')]):
    table,field = table_field.split('.')
    a = classified_var_meta[(classified_var_meta.TABLE == table)&(classified_var_meta.FIELD == field)]
    if a.shape[0] != 0:
        table_final = pd.concat([table_final,a],axis=0)
        schema_final.append(table_field)
print(f"최종 선택 Schema 및 Meta 정보")
print(table_final.to_markdown())


# 우리가 사용하는 TABLE & FIELD 범위 내에 있는 애들만 schema linking 결과에 포함되도록
field_list = schema_final.copy()

# schema linking 결과 내에 존재하는 Entity들에 대한 Desc를 검색하여 저장
target_vars_ = table_final.copy()
    
search_result = value_description[value_description.Field.isin(target_vars_.FIELD)]

# 4. SQL 구문 생성 및 데이터 조회

### 4-1. SQL 구문 생성

In [25]:
llm_sql_llama3 = OpenAI(
api_key = llama_api_key,
base_url = "https://api.llama-api.com"
)

In [26]:
## 수정 ## 프롬프트 부분부분 바뀐게 많아 이 블럭 전체 뜯어서 교체해주면 돼

schema_link = f"[Table & Fields]{schema_final}"

temp = f"""
현업의 요청사항을 처리하기 위해 준비한 참고자료는 아래와 같아.

** 참고자료 **

    ### 1. 원본 질문 (Original Question)
    {question}

    ### 2. 정제된 질문 (Refined question)
    {refined_question}

    ### 3. Schema linking 결과(Schema linking) 
    {schema_link}

    ### 4. Table Description (Description of necessary Tables)
    {table_description[table_description.Table.isin(target_vars_.TABLE.unique())].to_markdown()}

    ### 5. Field & Value Description (Description of the field and values)
    {search_result[['Field','Field Description','Unique_Value']].to_markdown()}

위의 정보들을 모두 활용할 필요는 없어. 특히, Schema linking의 경우 SQL 작성에 불필요한 부분은 제외해도 좋아.
SQL 작성 시 주의사항은 아래와 같아.

** 주의사항 **

    ### 1. 날짜 format은 YYYY-MM-DD야.
    ### 2. Python 내장 Sqlite3 DB 기준으로 쿼리를 작성해줘.
    ### 3. Please be careful when writing the FIELD name. Be sure to write with reference to the FIELD name within the Field & Value Description range.
    ### 4. String Type FIELD의 경우, 조회하는 Text와 정확히 일치하지 않을 수 있으므로, LIKE %Text% 구문을 활용해줘.
    ### 5. Be sure to include SELECT and FROM, which are the basic components of SQL query statements.
    
** Always output only your answer**
"""

##### 24-10-03 조윤재 수정 > llama3로 변경

test = llm_sql_llama3.chat.completions.create(
model="llama3.1-70b",
messages=[
    {"role": "system", "content": "너의 역할은 현업의 요청에 따라 데이터를 추출하는 SQL 쿼리를 작성하는 Assistant야."},
    {"role": "user", "content": temp}
],)
test = test.choices[0].message.content
test = test.split('sql')[-1].split("```")[0]

In [27]:

# SQL 생성 시 주의해야 할 사항은 아래와 같아.

# ### 주의사항
# 1. Table 및 Field Description에 존재하는 Field만 활용해줘. 존재하지 않는 가상의 Field를 만들어내지 말아줘.

# 2-1. 문자열 기반의 Field(ex. 그룹명, 기업명, 자재명, 사람이름 등)의 경우, 요청사항에서의 명칭과 DB 내 값이 정확히 일치하지 않을 수 있으므로, LIKE %특정대상% 구문을 활용해줘.
# 2-2. 예시 : XYLENE --> %XYLENE%, 엘지화학 --> %엘지화학%, 대한민국 --> %대한민국% 등

# 3. SQL 구문은 Python sqlite3 기준으로 작성해줘.
# 4. 추가적인 정보를 제외한 SQL Query문만 대답해줘.
# 5. 데이터 집계 시, 특별한 요청이 없다면 월(Month) 단위를 기본으로 해줘.
# 1-1. SQL 구문 내에서 특정 Field 활용에 앞서, Field Description을 참고하여 Table과 Field 간의 관계를 고려해줘.
# 1-2. Description에서 실제로 Table 내에 존재하는 것으로 확인되는 Field를 활용해줘. 존재하지 않는 Field를 만들어내지 말아줘.

# 2-1. 날짜 관련 field의 경우, field 형태가 모두 yyyy-mm-dd로 통일되어 있어. 조건문 등 작성 시, 이 점을 참고해줘.
# 2-2. 날짜의 경우, schema mapping에서 필요한 테이블의 날짜 Field가 누락되었을 가능성이 있어. 아래의 3-3. Hint를 참고해줘.
# 2-3. Hint : YNCC_BI_TABLE의 날짜 FIELD는 `날짜`야

# 3-1. 문자열 기반의 특정 대상(그룹명, 기업명, 자재명, 사람이름 등)의 경우, 요청사항에서의 명칭과 DB 내 값이 정확히 일치하지 않을 수 있으므로, LIKE %특정대상% 구문을 활용해줘.
# 3-2. 예시 : XYLENE --> %XYLENE%, 엘지화학 --> %엘지화학%, 대한민국 --> %대한민국% 등

# 4. 진행 상황, 완료 여부 등 대상의 상태를 묻는 질문은 CASE WHEN 구절을 활용하여 세부적인 진행내역을 볼 수 있도록 해줘.

# 5. SQL의 조건절(WHERE) 작성 시, 주석 처리의 용이성을 위해 True 구문으로 시작해줘 (1=1).

# 6. field 호출 시, 요청사항 내 Entity를 활용하여 적절하게 이름 변경(AS SOMETHING)을 해줘. 한국어로 작성해줘.

# 7-1. SQL 구문은 Python sqlite3 기준으로 작성해줘.


# 8. 추가적인 정보를 제외한 SQL Query문만 대답해줘.

# 9. 조회되는 결과 상에서 금액, 수량, 차액 등 중요한 정보의 값이 nan, None 등 비어있는 경우가 없도록 추가 쿼리를 작성해줘.

# 10-1. 신규거래업체는 조회 기간 내에 첫 거래가 발생한 업체를 의미해. 따라서, 조회 기간보다 과거에는 거래 이력이 없어야 해.
# 10-2. 거래정지업체는 조회 기간 이전에는 거래가 있지만 이후 시점부터는 거래가 없어진 업체를 의미해. 따라서 조회 기간보다 과거에만 거래 이력이 있어야 해.
# 10-3. 이를 위해 WITH문을 사용하는 방법도 고려해줘.

# 11. 구매금액의 추이 등 시계열 조회, 집계 등이 필요한 경우, 요청사항에 시간 단위가 포함되어 있지 않다면, 월 단위를 기준으로 해줘.


# 12-1. 실제 데이터베이스에 요청하여 결과를 얻을 수 있는 쿼리문으로 끝까지 작성해줘.
# 12-2. 예를 들면, 너가 만든 SQL 구문에 WITH절만 있다면 쿼리문 실행 시 오류가 발생할거야. 스스로 만든 쿼리에 대해 검토해보고 부족한 부분을 보강해줘.

### 4-2. 결과 종합

In [ ]:
print(f'refined question is :{refined_question}')
print('-----------------------------------------------')
print(f'작성된 SQL 결과 :{test}')
get_data = pd.read_sql(test,engine)
print('조회된 데이터 결과')
print(get_data.to_markdown())
print('-----------------------------------------------')

In [24]:
# TO-DO list (TBD)
# 1-1. 코드북 작업 / ex. C001 -> 칠레 / 사용자의 질문이 C001 판매실적 알려줘 일 때, 제품 FIELD로 연결지을 bridge 기능 필요함
# 1-2. 현재의 비즈니스 용어 사전은 (매출, 실적, 판매금액) --> Sales Amount로 연결짓는 상위 level 사전임

# 2. 삼섬 knowledge 9단계 구성 (혹은 로직의 명칭) 확인
# 3. SQL 템플릿 vector DB 구축 > GPT-4o 기반일 때에는 템플릿의 복잡도가 문제되지 않았음, 현재는 구문 복잡도 상승 시 답변 완결성이 보장되지 못함

# 4. vector DB 보강 > 현재 query, vocab 굉장히 simple, 모델의 일반화를 목적으로 논리적 흐름 기반 아키텍처
# 5. 답변 시간 제한 고려 필요
# 6. 답변이 틀릴 때 오류 메세지보다는 예외처리로 빠져서 안내하도록
# 7. 정답에 대한 사용자의 응답을 기반으로 vector DB 통에 저장하는 기능 (성공DB, 실패DB --> 운영자가 정보 생성 및 업로드)
# 8. 분석(코멘트), 시각화 --> TBD
# 9. 데이터 범위 늘어나게 될 때 (현행 주문 PoC --> 반품 등 범위 확장 시) 아키텍처 어떻게 가져갈지
# (안) : 주문 T2A, 반품 T2A 개별 구성 및 상위 계층에서 질문의 의도를 분류하여 모델 호출하는 로직



# 현행 Logic
# 사용자 질문 (러프한 질문 입력): 매출 알려줘
# 재질의 (불충분한 정보는 LLM의 정확한 답변을 기대할 수 없음): 2014년도 A사업부 매출 알려줘
# few-shot DB 유사 사례 검색 (일치, 유사, 없음): 정답지 성격의 data 기반 + LLM (NER & mapping 기반) 후보 FIELD 식별
# ex. NER & mapping (LLM으로 추출한 엔티티: [2014년, A사업부, 매출]-->META 내 필드: [Date, Org, Amount])
# SQL 생성 (정확한 질문, 부족함 없는 후보 FIELD, FIELD들의 의미, FIELD들의 값 예시 주어지면 SQL은 잘 만들어짐, 단 llama3.1 70b는 복잡도에 따른 답변 성능 issue 존재)



## 그래프그리기 

In [25]:
def contains_react_words(text, react_words):
    return any(word in text for word in react_words)

In [ ]:
def remove_markers(text, lst):
    # Remove the markers ```, ```python and ~~~~
    for replace_target in lst:
        text = text.replace(replace_target, "")
    return text.strip()


In [ ]:
plot_input = f"""Question : {refined_question}\n
                Answer : {get_data.to_markdown()}
                """

In [ ]:
plot_words = ['그래프','그레프','플롯']
if contains_react_words(plot_input, plot_words):
    # 시각화 생성
    temp_prompt = f"""
    Analyze the "Question" given below, and use the data in the "Answer" to create a plot that is easy for the user to understand.
    Suggest the most appropriate plot type and draw it using Python's matplotlib. 
    To display it in Streamlit, assign it to 'fig' at the end of the code, but do not include 'st.pyplot(fig)'.
    \n
    {plot_input}
    \n
    ** Output Flow **
    1) package import - import matplotlib.pyplot as plt, import matplotlib as mpl, from matplotlib import font_manager, import pandas as pd
    2) data generate
    3) change data to dataframe
    4) assign fig and make plot

    ** If a plot is not needed, print None only **
    ** Ensure to include the part for importing necessary packages **
    ** Generate the code only without additional explanations **
    """
    
    messages = [HumanMessage(content=temp_prompt)]
    return_dict = llm_tmpt0(messages)
    return_text = return_dict.content

    ## hardcoded
    if contains_react_words(return_text, ['```python', '```']):
        return_text = remove_markers(return_text, lst = ['```python', '```'])
    if contains_react_words(return_text, ['st.pyplot(fig)']):
        return_text = return_text.replace('st.pyplot(fig)', '')
        
    print(return_text)
    # Create a dictionary to use as the namespace
    namespace = {}
    # Execute the code in the context of the provided namespace
    exec(return_text, namespace)
    # fig 할당
    fig = namespace.get('fig')

In [ ]:
return_text

In [ ]:
fig